# Evaluating Janus (v 0.1.2)
This notebook allows us to evaluate Janus and output relevant metrics for documentation.

In [10]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

# Transforming image to standardized numpy array:
def to_array(image_path):
	img_arr=cv2.imread(image_path)
	img_arr=cv2.resize(img_arr,(224,224))
	test_input=np.array([img_arr])
	test_input=test_input/225.0
	return test_input

# Printing the result:
def get_result(result):
	if result == 0: 
		return "ABSENT"
	if result == 1: 
		return "EVEN"
	if result == 2: 
		return "WEIGHTED"

# Reconstructing janus:
reconstructed_janus = load_model("../janus")

In [11]:
# Testing on a single image:
image_path = input("Input image path: ")

In [14]:
# Checking the result:
result = reconstructed_janus.predict(to_array(image_path)).argmax(axis=1)[0]
print(image_path + " is " + get_result(result))

1/1 [==============================] - 0s 235ms/step
/home/juris/local/199_ariadne/janus/final-dataset/test/even/ZDNetPopup.png is EVEN


## Validation

Testing using data that was part of the training data set.

In [25]:
val_path="../final-dataset/val"

val_file = []
val_arr= []
val_ref = []
val_pred = []

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder 

    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        val_file.append(image_path)
        val_arr.append(to_array(image_path))
        val_ref.append(folder)
        val_pred.append(get_result(reconstructed_janus.predict(to_array(image_path)).argmax(axis=1)[0]))

1/1 [==============================] - 0s 154ms/step


In [26]:
data = {'File Path': val_file,
        'Array': val_arr,
		'Label': val_ref,
		'Prediction': val_pred}

df_val = pd.DataFrame(data)
print(df_val)
df_val.to_csv('validation.csv', index=False)

                                            File Path  \
0    ../final-dataset/val/weighted/CostaNewsPopup.png   
1   ../final-dataset/val/weighted/BloombergPopup1.png   
2          ../final-dataset/val/weighted/CBCPopup.png   
3       ../final-dataset/val/weighted/AbantePopup.png   
4    ../final-dataset/val/absent/BradfordianPopup.png   
5   ../final-dataset/val/absent/BusinessReviewPopu...   
6             ../final-dataset/val/absent/AAPopup.png   
7   ../final-dataset/val/absent/CountryLivingPopup...   
8     ../final-dataset/val/even/PressGazettePopup.png   
9     ../final-dataset/val/even/HistoryTodayPopup.png   
10  ../final-dataset/val/even/TheIsleOfThanetNewsP...   
11  ../final-dataset/val/even/AlgarveDailyNewsPopu...   

                                                Array     Label Prediction  
0   [[[[0.79555556 0.79555556 0.79555556], [0.7822...  weighted   WEIGHTED  
1   [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...  weighted   WEIGHTED  
2   [[[[1.13333333 0.475555

## Testing

Testing using data not from the training data set.

In [31]:
test_path="../final-dataset/test"

test_file = []
test_arr= []
test_ref = []
test_pred = []

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder 

    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        test_file.append(image_path)
        test_arr.append(to_array(image_path))
        test_ref.append(folder)
        test_pred.append(get_result(reconstructed_janus.predict(to_array(image_path)).argmax(axis=1)[0]))

1/1 [==============================] - 0s 159ms/step


In [32]:
data = {'File Path': test_file,
        'Array': test_arr,
		'Label': test_ref,
		'Prediction': test_pred}

df_test = pd.DataFrame(data)
print(df_test)
df_test.to_csv('testing.csv', index=False)

                                           File Path  \
0        ../final-dataset/test/weighted/WSJPopup.png   
1  ../final-dataset/test/weighted/WashingtonTimes...   
2      ../final-dataset/test/absent/WeForumPopup.png   
3  ../final-dataset/test/absent/WorkinghamPaperPo...   
4  ../final-dataset/test/even/WashingtonPostPopup...   
5          ../final-dataset/test/even/ZDNetPopup.png   

                                               Array     Label Prediction  
0  [[[[1.05777778 1.05777778 1.05777778], [1.0577...  weighted       EVEN  
1  [[[[0.87111111 0.43111111 0.        ], [0.88  ...  weighted   WEIGHTED  
2  [[[[1.11111111 1.10666667 1.10666667], [1.1111...    absent   WEIGHTED  
3  [[[[0.22222222 0.20888889 0.21333333], [0.3111...    absent     ABSENT  
4  [[[[0.62666667 0.50666667 0.37333333], [0.6355...      even       EVEN  
5  [[[[0. 0. 0.], [0. 0. 0.], [0. 0. 0.], [0. 0. ...      even       EVEN  
